# Data exploration

In [26]:
import pandas as pd

## Data import
The original dataset is > 2 million rows. This is too much data for my laptop to process at once, so as a data pre-processing step I have reduced this down to 50,000 rows by random sampling. 

An alternative approach to random sampling would be to eg. take all the loans issued in a chosen calendar year as the chosen sample. The issue with this approach is that we know from the client that "In the recent years, the share of defaulted loans issued on the platform has increased significantly" - so selecting loans from only a certain time period may introduce bias. 

In sampling this data I am assuming that each row represents a unique loan: ie. that a loan will not appear twice in the dataset with different loan_status values. As the id variable is anonymised as null in all cases this is a prudent assumption. 

Let's start by getting an overview of the shape of the data and the meaning of the columns: 

In [30]:
orig_df = pd.read_csv("../../data/processed/loan_sampled_50000.csv", low_memory = False)
df = orig_df.copy(deep = True)

In [31]:
df.shape

(50000, 145)

In [33]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,6200,6200,6200.0,36 months,10.41,201.26,B,B3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,19600,19600,19600.0,60 months,25.69,583.25,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,16000,16000,16000.0,60 months,11.99,355.84,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,16400,16400,16400.0,36 months,15.77,574.72,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,7100,7100,7100.0,36 months,14.49,244.36,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df_data_dictionary = pd.read_excel("../../data/raw/LCDataDictionary.xlsx").dropna()

In [41]:
df_data_dictionary.style.set_properties(subset=["Description"], **{"width": "800px"})

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower is now delinquent.
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan application
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by the borrower during registration.
5,annual_inc_joint,The combined self-reported annual income provided by the co-borrowers during registration
6,application_type,Indicates whether the loan is an individual application or a joint application with two co-borrowers
7,avg_cur_bal,Average current balance of all accounts
8,bc_open_to_buy,Total open to buy on revolving bankcards.
9,bc_util,Ratio of total current balance to high credit/credit limit for all bankcard accounts.


## Add target variable 
As per the client:

Default client is defined as one with loan_status variable taking on the following levels:
- Charged off
- Default
- Does not meet the credit policy. Status: Charged Off
- Late (31-120 days)

We are left to construct our own definition for a non-default client. 

There are numerous approaches we could take on this:

1. Take all clients who do not meet the default definition as non-default. The drawback to this is that it includes clients in statuses that may later lead to default. A client in the status "current", "late (16-30 days)", or "in grace period" may later default on the loan: we have no way of predicting the future! 

2. Take all clients who we know have paid off their loan as non-default. From the business perspective this is the safer choice: we can guarantee the status of these clients and we are not introducing bias from customers that may default later. 

So, we define a non-default client as one with the loan_status variable taking on the following levels:
- Fully paid
- Does not meet the credit policy. Status: Fully paid 

What do we do with clients that are in an in-between state (ie. those we cannot name as a definite default or non-default)? Our task is to predict clients that default. Working on the hypothesis that there will be noticeable differences in characteristics between default and non-default clients, the dataset of clients that are neither will contain clients of both characteristics. To reduce any confusion this may introduce in the predictive stage - and for ease of analysis - we will simply remove these clients for now. 

In [60]:
default_client_values = ["Charged Off", "Default", "Does not meet the credit policy. Status:Charged Off", 
                        "Late (31-120 days)"]
non_default_client_values = ["Fully Paid", "Does not meet the credit policy. Status:Fully Paid"]

In [42]:
df["loan_status"].value_counts()

Fully Paid                                             23164
Current                                                20335
Charged Off                                             5671
Late (31-120 days)                                       477
In Grace Period                                          212
Late (16-30 days)                                         79
Does not meet the credit policy. Status:Fully Paid        43
Does not meet the credit policy. Status:Charged Off       18
Default                                                    1
Name: loan_status, dtype: int64

In [68]:
def drop_inbetween_clients(df):
    target_values = default_client_values + non_default_client_values
    return df[df["loan_status"].isin(target_values)]

In [70]:
df = drop_inbetween_clients(df)
df.shape

(29374, 146)

By dropping the clients whose statuses we cannot infer, we reduce the size of our dataset by around 40%. 

Next, let's add the target variable. We'll say that a default client is 1 (the class we're trying to predict) and a non-default client is 0.

In [71]:
def add_target_variable(df):
    df["target"] = df["loan_status"].isin(default_client_values)
    return df

In [74]:
df = add_target_variable(df)
df["target"].value_counts()

False    23207
True      6167
Name: target, dtype: int64

Adding the target variable, we can see we have a class imbalance: roughly 4x as many non-default clients as default. This may cause problems later: for now, let's continue with our data analysis. 